In [1]:
# Imports
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pickle
from libHanSolo import unpacker
from matplotlib import pyplot

In [2]:
# Constants
INDEX_ORDER = [(x, x + 1024, x + 2048) for x in range(0, 1024)]
INPUT_SHAPE = (32, 32, 3)

In [3]:
def DataToImage(pixel_values):
    ordered = pixel_values[INDEX_ORDER]
    return np.array(ordered.reshape((32,32,3)))

def BatchToImages(batch):
    images = unpacker.get_value(batch, 'data')
    return np.array([DataToImage(image) for image in images])

def LabelToVector(label_value):
    arr = np.zeros(10)
    arr[label_value] = 1.0
    return arr

def BatchToLabelVector(batch):
    labels = unpacker.get_value(batch, 'labels')
    return np.array([LabelToVector(label) for label in labels])

def SetupData():
    batches = [unpacker.unpickle(f'./data/data_batch_{index}') for index in range(1, 6)]
    test = unpacker.unpickle(f'./data/test_batch')
    X_train = np.concatenate([BatchToImages(batch) for batch in batches])
    X_test = BatchToImages(test)
    y_train = np.concatenate([BatchToLabelVector(batch) for batch in batches])
    y_test = BatchToLabelVector(test)
    return [X_train, y_train, X_test, y_test]

In [4]:
[X_train, y_train, X_test, y_test] = SetupData()
print(f'{X_train.shape} {X_test.shape} {y_train.shape} {y_test.shape}')

(50000, 32, 32, 3) (10000, 32, 32, 3) (50000, 10) (10000, 10)


In [103]:
def ResizeImage(image_set, size):
    for image in image_set:
        yield tf.image.resize(image, size)
    
    

In [104]:
X_train_generator = ResizeImage(X_train, [224, 224])

In [23]:
model = keras.applications.resnet50.ResNet50(weights="imagenet")

In [105]:
inputs = keras.applications.resnet50.preprocess_input(X_train_generator)
pred = model.predict(X_train_generator)
next(X_train_generator).shape

## Problemer med at have alt i mem paa samme tid

ValueError: in user code:

    /opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1147 predict_function  *
        outputs = self.distribute_strategy.run(
    /opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1122 predict_step  **
        return self(x, training=False)
    /opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer.py:927 __call__
        outputs = call_fn(c